In [1]:
import pandas as pd
import numpy as np

In [2]:
#leer data original
db = pd.read_csv('../data/molinetes_historico_2018.csv',sep=';')

In [3]:
#filtar febrero
db = db.loc[db.PERIODO == 201802,:] 

In [4]:
#seleccionar columnas relevantes
db = db.reindex(columns = ['ID','FECHA', 'DESDE', 'LINEA', 'ESTACION','TOTAL'])

In [5]:
#obtener tipo de dia y hora
db['FECHA'] = pd.to_datetime(db.FECHA + ' ' + db.DESDE,format = '%d/%m/%Y %H:%M:%S')
db['hora'] = db.FECHA.map(lambda x: x.hour) 
db['fin_de_semana'] = db.FECHA.map(lambda x: x.weekday()) < 5

In [6]:
#eliminar variables innecesarias
db = db.drop(['FECHA','DESDE'],axis=1)

In [7]:
#visualizar el resultado
db.head()

,ID,LINEA,ESTACION,TOTAL,hora,fin_de_semana
1025451,42,LINEA_A,ACOYTE,0,5,True
1025452,37,LINEA_A,ALBERTI,1,5,True
1025453,75,LINEA_A,CARABOBO,3,5,True
1025454,40,LINEA_A,CASTRO BARROS,6,5,True
1025455,82,LINEA_A,FLORES,1,5,True


In [8]:
#finalmente podemos procesar los datos
dataInsumo = db.groupby(['fin_de_semana','hora','LINEA','ID',]).mean().reset_index()
dataInsumo.head()

,fin_de_semana,hora,LINEA,ID,TOTAL
0,False,6,LINEA_A,31,2.239130
1,False,6,LINEA_A,32,2.705882
2,False,6,LINEA_A,33,2.859649
3,False,6,LINEA_A,34,2.744681
4,False,6,LINEA_A,35,8.157143


In [9]:
#cambiar la ultima columnad de nombre a promedio, sumando la palabra promedio a los nombres hasta el ultimo exclusive  
dataInsumo.columns = dataInsumo.columns[:-1].tolist() + ['promedio']
dataInsumo.head()

,fin_de_semana,hora,LINEA,ID,promedio
0,False,6,LINEA_A,31,2.239130
1,False,6,LINEA_A,32,2.705882
2,False,6,LINEA_A,33,2.859649
3,False,6,LINEA_A,34,2.744681
4,False,6,LINEA_A,35,8.157143


In [10]:
#luego agregamos los quintiles, pero queremos que comiencen desde 1 no desde 0
#(la convención en estadística es diferente a Python)
dataInsumo['q'] = pd.qcut(dataInsumo.promedio,5,labels=False) + 1 #sumamos 1 a toda la columna de quintiles
dataInsumo.head()

,fin_de_semana,hora,LINEA,ID,promedio,q
0,False,6,LINEA_A,31,2.239130,1
1,False,6,LINEA_A,32,2.705882,1
2,False,6,LINEA_A,33,2.859649,1
3,False,6,LINEA_A,34,2.744681,1
4,False,6,LINEA_A,35,8.157143,2


In [11]:
#guardar datos en csv para que los tome otro proceso luego
dataInsumo.to_csv('../data/dataInsumo.csv',index=False)

In [12]:
#tambien se puede guardar en excel
dataInsumo.to_excel('../data/dataInsumo.xlsx',index=False)